# Feature Analysis of Website Imagery Content

## Imports

In [2]:
import numpy as np
import pandas as pd
import pprint
import os
import pymongo
import random
import re
import nltk
from nltk.corpus import stopwords
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
from pymongo import MongoClient
from PIL import Image
from gridfs import GridFS

## Inputs

In [5]:

client = MongoClient(open('../../db.txt', encoding='UTF-8').read())
db = client.knowledge

## Data Collection

In [6]:
labels = db.labels
ct = labels.count_documents({'image':{'$ne':None}})
print(ct)

firstImage = labels.find_one({'image':{'$ne':None}})
print(firstImage)
# Image.frombytes(mode='RGB',size=(1000,760),data=firstImage['image'])

# images = [label['image'] for label in labels.find({'image':{'$ne':None}})]
# print(len(images))

114
{'_id': ObjectId('5bd5202204c0591b776aeb2e'), 'date': datetime.datetime(2018, 10, 28, 2, 34, 10, 295000), 'url': 'https://id.pausd.org/portal/p/applications', 'label': 'y', '__v': 0, 'image': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03 \x00\x00\x02X\x08\x06\x00\x00\x00\x9av\x82p\x00\x00\x0cpIDATx\x9c\xed\xd7A\r\xc0 \x00\xc0@\xc0\xbf\xd3%\x04\x07\x9b\x8b\xf2\xd8\x9d\x82~;\x9f}\xde\x01\x00\x00\x10X\xb7\x03\x00\x00\x80\xff0 \x00\x00@\xc6\x80\x00\x00\x00\x19\x03\x02\x00\x00d\x0c\x08\x00\x00\x901 \x00\x00@\xc6\x80\x00\x00\x00\x19\x03\x02\x00\x00d\x0c\x08\x00\x00\x901 \x00\x00@\xc6\x80\x00\x00\x00\x19\x03\x02\x00\x00d\x0c\x08\x00\x00\x901 \x00\x00@\xc6\x80\x00\x00\x00\x19\x03\x02\x00\x00d\x0c\x08\x00\x00\x901 \x00\x00@\xc6\x80\x00\x00\x00\x19\x03\x02\x00\x00d\x0c\x08\x00\x00\x901 \x00\x00@\xc6\x80\x00\x00\x00\x19\x03\x02\x00\x00d\x0c\x08\x00\x00\x901 \x00\x00@\xc6\x80\x00\x00\x00\x19\x03\x02\x00\x00d\x0c\x08\x00\x00\x901 \x00\x00@\xc6\x80\x00\x00\x00\x19\x03\x02\x00\x00d\x0c\x08\x0

## Feature Extraction

In [10]:
for image in images:
    img = Image.open(image)
    print(dir(img))

ValueError: embedded null byte

### Grabbing Images

In [ ]:
image_url = 
image_data = BytesIO(requests.get(image_url).content)
image_obj = Image.open(image_data)
raw_image_data = image_obj.tostring()